## Langchain Demo: AI Agents on AWS Bedrock

This notebook demonstrates a proof of concept for an Agentic & Tool approach for integrating AI services with the `getout_of_text3` module.


It's easy to leverage [`langchain`](https://python.langchain.com/docs/tutorials/agents/) in python for creating AI agents with tools, and provides support across various cloud providers.
- On this notebook, we leverage [AWS Bedrock](https://aws.amazon.com/bedrock/) for access to the OpenAI open-sourced GPT model [`gpt-oss-120b`](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-openai.html) and 
- We bind [tools](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html) to our AWS Bedrock model to invoke custom workflows, namely *to summarize KWIC filtered hits from our DIY SCOTUS Corpus*.
- Make sure to configure your **AWS named profile** in `~/.aws/credentials` file with IAM permissions to invoke models on AWS

In [175]:
# Ensure your AWS credentials are configured
import langchain
from langchain.chat_models import init_chat_model
import pandas as pd
import getout_of_text_3 as got3

In [ ]:
#model_id='us.deepseek.r1-v1:0' # does not work with tools implementation
#model_id='us.meta.llama4-maverick-17b-instruct-v1:0' # 1M
#model_id='global.anthropic.claude-sonnet-4-5-20250929-v1:0' # 200K
#max_tokens=64000 # for anthropic.claude-sonnet-4-5-20250929-v1:0
#model_id='amazon.nova-pro-v1:0'
#max_tokens=10000
#model_id='us.amazon.nova-premier-v1:0'
#max_tokens=32000
model_id='openai.gpt-oss-120b-1:0' # 128,000 max token limit
max_tokens=128000 # for openai.gpt-oss-120b-1:0

In [91]:
model = init_chat_model(model_id, 
                        model_provider="bedrock_converse",
                        credentials_profile_name='atn-developer',
                        max_tokens=max_tokens # maximum for bedrock converse
                        )

In [92]:
query = "Hi! What is the capital of Portugal?"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'The capital of Portugal is **Lisbon**.'

## DIY SCOTUS Corpus Query Tool & Example

- Using `got3` to filter on keywords in a DIY SCOTUS Corpus, via extracting text in PDFs downloaded from **Library of Congress** collection on the [United States Reports](https://www.loc.gov/collections/united-states-reports/).
- Corpus is stored as a **dictionary** of **dataframes**, where each key is a **volume number** and each dataframe contains the cases in that volume.
    - the `case_id` is a concatenated volume and page number (i.e. `329001` is volume 329, page 1). This is also a schema for saving PDF downloads locally. See details at [the U.S. Report page](https://www.supremecourt.gov/opinions/USReports.aspx).

        ```json
        {"329": {DataFrame}, 
        "330": {DataFrame}, ..., 
        "570": {DataFrame}
        }
        ```
        ```text
        case_id	text
        0	570729	OCTOBER \nTERM, 2012 \n729 \nSyllabus \nSEKHAR...
        1	570338	338 \nOCTOBER \nTERM, 2012 \nSyllabus \nUNIVER...
        2	570099	OCTOBER \nTERM, 2012 \n99 \nSyllabus \nALLEYNE...
        ```

In [94]:
# read pdf scotus files
df = pd.read_json("loc_gov.json", lines=True)

df['key'] = df['filename'].apply(lambda x: x.split('usrep')[1][:3])
df['subkey'] = df['filename'].apply(lambda x: x.split('usrep')[1].split('.pdf')[0])

# Create a dictionary to hold the DataFrame contents
df_dict = {}

for _, row in df.iterrows():
    if row['key'] not in df_dict:
        df_dict[row['key']] = {}
    df_dict[row['key']][row['subkey']] = row['content']

# format scotus data for getout_of_text_3, similar to COCA keyword results
db_dict_formatted = {}
for volume, cases in df_dict.items():
    # Create a DataFrame for each volume with case text
    case_data = []
    for case_id, case_text in cases.items():
        case_data.append({'case_id': case_id, 'text': case_text})
    db_dict_formatted[volume] = pd.DataFrame(case_data)


In [97]:
sorted_keys = sorted(db_dict_formatted.keys(), key=lambda x: int(x), reverse=False)
print(sorted_keys)
db_dict_formatted['570'].head()

['329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423', '424', '425', '426', '427', '428', '429', '430', '431', '432', '433', '434', '435', '436', '437', '438', '439', '440', '441', '442', '443', '444', '445', '446', '447', '448', '449', '450', '451', '452', '453', '454', '455', '456', '457', '458', '459', '460', '461', '462', '463', '464', '465', '466', '467', '468', '469', '470', '471'

,case_id,text
0,570729,"OCTOBER \nTERM, 2012 \n729 \nSyllabus \nSEKHAR..."
1,570338,"338 \nOCTOBER \nTERM, 2012 \nSyllabus \nUNIVER..."
2,570099,"OCTOBER \nTERM, 2012 \n99 \nSyllabus \nALLEYNE..."
3,570529,"OCTOBER \nTERM, 2012 \n529 \nSyllabus \nSHELBY..."
4,570048,"48 \nOCTOBER \nTERM, 2012 \nSyllabus \nMARACIC..."


## Creating Tools for AI Agents

The following cell outlines two custom toolsets for the `got3` project. Neither is a definitive or factual summary of course, however this conceptual approach aims to:
- leverage the latest trend in AI technologies, moving beyond generic chatbots to an **agentic approach**.

### Tool Summary

1. `ScotusAnalysisTool` which takes a keyword as an argument and attempts to filter local corpus files, and then invoke a model tool with the results.
2. `ScotusFilteredAnalysisTool` which allows the user to provide the filtered local corpus extract (helpful to remove stopwords, false positive KWIC hits, etc) to then invoke a model tool with the results.


In [ ]:
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from typing import Optional, Type, Dict, Any, Union, List
import json
import asyncio
import re
from pathlib import Path


# =========================a===================================================
# ORIGINAL live-search tool (now simplified for notebook-friendly execution)
# ============================================================================
class ScotusAnalysisInput(BaseModel):
    """Input for SCOTUS case analysis tool (performs a fresh keyword search)."""
    keyword: str = Field(description="The keyword to search for and analyze in SCOTUS cases")
    analysis_focus: Optional[str] = Field(
    default="general", 
    description="Focus of analysis: 'general', 'evolution', 'judicial_philosophy', or 'custom'"
    )

class ScotusAnalysisTool(BaseTool):
    """Tool that SEARCHES the SCOTUS corpus then analyzes.
    NOTE: For pre-filtered JSON results, use ScotusFilteredAnalysisTool instead.

    Implementation note (merged sync/async):
    - We removed the prior _sync_run/_arun split + nest_asyncio hack.
    - _run always works in notebooks (Jupyter event loop) using the sync model.invoke.
    - A lightweight _arun delegator is kept for LangChain compatibility but just calls the sync core.
    """
    name: str = "scotus_analysis"
    description: str = (
        "Analyzes SCOTUS cases for a given keyword after performing an internal search. "
        "Do NOT provide pre-filtered results to this tool."
    )
    args_schema: Type[BaseModel] = ScotusAnalysisInput
    model: Any = Field(exclude=True)
    db_dict_formatted: Any = Field(exclude=True)

    def __init__(self, model, db_dict_formatted, **kwargs):
        super().__init__(**kwargs)
        self.model = model
        self.db_dict_formatted = db_dict_formatted

    # Public entry point (sync) ------------------------------------------------
    def _run(self, keyword: str, analysis_focus: str = "general") -> str:  # noqa: D401
        try:
            return self._execute(keyword, analysis_focus)
        except Exception as e:
            error_msg = f"Error analyzing SCOTUS results: {e}"
            print(f"❌ TOOL(search): {error_msg}")
            return error_msg

    # Async compatibility (simply defers to sync to avoid loop issues in notebooks)
    async def _arun(self, keyword: str, analysis_focus: str = "general") -> str:  # noqa: D401
        return self._run(keyword, analysis_focus)

    # Internal core (shared) --------------------------------------------------
    def _execute(self, keyword: str, analysis_focus: str) -> str:
        print(f"🔍 TOOL(search): Searching SCOTUS database for keyword: '{keyword}'")
        import getout_of_text_3 as got3
        # Dynamic context window that auto-shrinks if prompt would exceed max_tokens
        base_context_words = 20
        context_words = base_context_words
        attempts = 0
        final_prompt = None
        results_dict = None
        volumes = []
        total_cases = 0
        while True:
            search_results = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=self.db_dict_formatted,
                case_sensitive=False,
                show_context=True,
                context_words=context_words,
                output="json"
            )
            results_dict = {k: v for k, v in sorted(search_results.items(), key=lambda item: int(item[0])) if v}
            if not results_dict:
                return f"No results found for keyword '{keyword}' in the SCOTUS database."
            total_cases = sum(len(cases) for cases in results_dict.values())
            volumes = list(results_dict.keys())
            prompt = self._build_prompt(results_dict, keyword, analysis_focus, volumes, total_cases)
            if len(prompt) <= max_tokens or context_words <= 5:
                final_prompt = prompt
                break
            # Need to shrink
            attempts += 1
            ratio = max_tokens / len(prompt)
            new_context_words = max(5, int(context_words * ratio * 0.9))  # 0.9 safety margin
            if new_context_words >= context_words:
                # No further shrink possible; accept risk of truncation
                print(f"⚠️ AUTO-SHRINK: Unable to further reduce context (floor reached or no progress). Using context_words={context_words}.")
                final_prompt = prompt
                break
            print(f"🪄 AUTO-SHRINK: prompt {len(prompt)} > max_tokens {max_tokens}. context_words {context_words} -> {new_context_words} (ratio {ratio:.3f}). Retrying...")
            context_words = new_context_words
        print(f"📊 TOOL(search): Found {total_cases} cases across {len(volumes)} volumes | context_words_used={context_words} | shrink_attempts={attempts}")
        print(f"🤖 TOOL(search): Sending {len(final_prompt)} characters to AI model for analysis")
        try:
            if len(final_prompt) > max_tokens:
                print(f"⚠️ TOOL(search): Prompt length {len(final_prompt)} still > max_tokens {max_tokens}; model/provider may truncate.")
            response = self.model.invoke([{"role": "user", "content": final_prompt}])
            content = getattr(response, 'content', str(response))
            print(f"✅ TOOL(search): Analysis complete, returning {len(content)} characters")
            return content
        except Exception as e:
            raise RuntimeError(f"Model invocation failed: {e}")

    def _build_prompt(self, results_dict, keyword, analysis_focus, volumes, total_cases) -> str:
        analysis_prompts = {
            "general": f"""
            Instructions:
            You are an AI Agent inside the open-source forensic linguistic tool `getout_of_text_3`.
            Analyze these SCOTUS case search results for the keyword \"{keyword}\" ONLY using the provided data.
            Data summary:
            - Volumes: {', '.join(sorted(volumes, key=int))}
            - Total case occurrences: {total_cases}
            Provide insights on:
            1. Temporal evolution
            2. Contextual variation
            3. Notable intra-dataset patterns (do NOT import outside knowledge)
            4. Interpretive themes relevant to ordinary meaning
            Results (truncated JSON): {json.dumps(results_dict, indent=2)}...
            """,
            "evolution": f"Focus on change over volumes for '{keyword}'.\nData: {json.dumps(results_dict, indent=2)}...",
            "judicial_philosophy": f"Assess patterns of usage that may hint at differing interpretive approaches for '{keyword}'. Use ONLY provided texts. Data: {json.dumps(results_dict, indent=2)}...",
            "custom": f"Comprehensive analysis for '{keyword}'. Use ONLY provided dataset. Data: {json.dumps(results_dict, indent=2)}..."
        }
        return analysis_prompts.get(analysis_focus, analysis_prompts["general"]).strip()

# ============================================================================
# Pre-filtered JSON analysis tool (enhanced: extraction strategies + debug metrics + token preflight)
# ============================================================================
class ScotusFilteredAnalysisInput(BaseModel):
    """Input for analyzing an already-filtered SCOTUS keyword JSON result set."""
    keyword: str = Field(description="Keyword being analyzed (for labeling only, not for searching).")
    results_json: Union[str, Dict[str, Any]] = Field(
        description="Pre-filtered JSON (or dict) output from got3.search_keyword_corpus AFTER user filtering."
    )
    analysis_focus: Optional[str] = Field(
        default="general", description="'general', 'evolution', 'judicial_philosophy', or 'custom'"
    )
    max_contexts: Optional[int] = Field(
        default=None, description="OPTIONAL cap on number of context snippets. If None/0 => include ALL."
    )
    return_json: bool = Field(
        default=False, description="If True, attempt to return structured JSON with reasoning_content, summary, etc."
    )
    extraction_strategy: str = Field(
        default="first",
        description="How to extract text from each occurrence: 'first' (first matching field), 'all' (all matching fields), 'raw_json' (embed entire JSON)."
    )
    debug: bool = Field(
        default=False, description="If True, prints and embeds debug metrics about extraction & token estimates."
    )

class ScotusFilteredAnalysisTool(BaseTool):
    """Analyze ONLY the supplied pre-filtered SCOTUS keyword result JSON.

    Enhanced with:
    - extraction_strategy (first|all|raw_json)
    - debug metrics (raw vs extracted char counts, estimated tokens)
    - token preflight rejection (early fail if would exceed model max_tokens)
    Simplified for notebooks (single sync execution core). Async call just delegates.
    """
    name: str = "scotus_filtered_analysis"
    description: str = (
        "Analyzes pre-filtered SCOTUS keyword search JSON (from got3) without performing any new retrieval."
    )
    args_schema: Type[BaseModel] = ScotusFilteredAnalysisInput
    model: Any = Field(exclude=True)

    def __init__(self, model, **kwargs):
        super().__init__(**kwargs)
        self.model = model

    # Normalization helper ----------------------------------------------------
    def _normalize_model_content(self, raw: Any) -> str:
        if isinstance(raw, str):
            return raw
        if isinstance(raw, list):
            parts = []
            for block in raw:
                if isinstance(block, str):
                    parts.append(block)
                elif isinstance(block, dict):
                    for key in ("text", "content", "value", "message"):
                        val = block.get(key)
                        if isinstance(val, str):
                            parts.append(val)
                            break
                    else:
                        parts.append(str(block))
                else:
                    parts.append(str(block))
            return "\n".join(parts)
        if isinstance(raw, dict):
            for key in ("text", "content", "value"):
                if key in raw and isinstance(raw[key], str):
                    return raw[key]
            return json.dumps(raw)
        return str(raw)

    # Public sync entry -------------------------------------------------------
    def _run(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "general",
        max_contexts: Optional[int] = None,
        return_json: bool = False,
        extraction_strategy: str = "first",
        debug: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        try:
            return self._execute(keyword, results_json, analysis_focus, max_contexts, return_json, extraction_strategy, debug)
        except Exception as e:
            msg = f"Error (filtered analysis): {e}"
            print(msg)
            return {"error": msg} if return_json else msg

    # Async delegator ---------------------------------------------------------
    async def _arun(
        self,
        keyword: str,
        results_json: Union[str, Dict[str, Any]],
        analysis_focus: str = "general",
        max_contexts: Optional[int] = None,
        return_json: bool = False,
        extraction_strategy: str = "first",
        debug: bool = False,
    ) -> Union[str, Dict[str, Any]]:
        return self._run(keyword, results_json, analysis_focus, max_contexts, return_json, extraction_strategy, debug)

    # Core logic --------------------------------------------------------------
    def _execute(self, keyword, results_json, analysis_focus, max_contexts, return_json, extraction_strategy, debug):
        if extraction_strategy not in {"first", "all", "raw_json"}:
            raise ValueError("extraction_strategy must be one of: 'first','all','raw_json'")
        results_dict = self._coerce_results(results_json)
        stats = self._compute_stats(results_dict, keyword, extraction_strategy)

        # Debug & metrics ------------------------------------------------------
        raw_json_str = json.dumps(results_dict, sort_keys=True)
        raw_chars = len(raw_json_str)
        # For metrics we still collect contexts unless raw_json mode
        if extraction_strategy == 'raw_json':
            extracted_contexts_for_metrics = self._sample_contexts(results_dict, max_contexts, 'all')
        else:
            extracted_contexts_for_metrics = self._sample_contexts(results_dict, max_contexts, extraction_strategy)
        extracted_chars = sum(len(c) for c in extracted_contexts_for_metrics)
        approx_tokens_raw = raw_chars / 4
        approx_tokens_extracted = extracted_chars / 4
        reduction_ratio = (extracted_chars / raw_chars) if raw_chars else 0
        if debug:
            print(f"🧪 DEBUG(filtered): raw_chars={raw_chars} extracted_chars={extracted_chars} reduction_ratio={reduction_ratio:.3f} raw≈{approx_tokens_raw:.0f}tok extracted≈{approx_tokens_extracted:.0f}tok strategy={extraction_strategy} limit={max_contexts}")

        # Build prompt ----------------------------------------------------------
        prompt = self._build_prompt(keyword, results_dict, stats, analysis_focus, max_contexts, return_json, extraction_strategy, debug, {
            'raw_chars': raw_chars,
            'extracted_chars': extracted_chars,
            'approx_tokens_raw': approx_tokens_raw,
            'approx_tokens_extracted': approx_tokens_extracted,
            'reduction_ratio': reduction_ratio,
            'extraction_strategy': extraction_strategy,
        })
        # Token preflight (char/4 heuristic) -----------------------------------
        approx_prompt_tokens = len(prompt) / 4
        if approx_prompt_tokens > max_tokens:
            msg = (f"Preflight rejection: prompt would exceed model max_tokens. approx_prompt_tokens={approx_prompt_tokens:.0f} > max_tokens={max_tokens}. "
                   f"Strategy='{extraction_strategy}' raw_tokens≈{approx_tokens_raw:.0f} extracted_tokens≈{approx_tokens_extracted:.0f}. Consider: lower max_contexts, switch to 'first', or filter upstream.")
            print(f"⚠️ {msg}")
            if return_json:
                return {
                    "error": "prompt_too_large",
                    "message": msg,
                    "keyword": keyword,
                    "total_contexts": stats['total_contexts'],
                    "extraction_strategy": extraction_strategy,
                    "raw_chars": raw_chars,
                    "extracted_chars": extracted_chars,
                    "approx_tokens_prompt": approx_prompt_tokens,
                }
            return msg

        print(f"🤖 TOOL(filtered): Sending {len(prompt)} chars (≈{approx_prompt_tokens:.0f} tok) to model (contexts: {stats['total_contexts']}) | strategy={extraction_strategy} return_json={return_json}")
        response = self.model.invoke([{"role": "user", "content": prompt}])
        raw = getattr(response, 'content', None)
        if raw is None and hasattr(response, 'text'):
            try:
                raw = response.text()
            except Exception:
                raw = response.text
        content = self._normalize_model_content(raw)
        if return_json:
            return self._postprocess_json(content, results_dict, stats)
        return content

    # ---------------- Internal helpers ----------------
    def _coerce_results(self, results_json: Union[str, Dict[str, Any]]) -> Dict[str, Any]:
        if isinstance(results_json, str):
            results_dict = json.loads(results_json)
        else:
            results_dict = results_json
        if not isinstance(results_dict, dict) or not results_dict:
            raise ValueError("results_json must be a non-empty dict or JSON string")
        return results_dict

    def _extract_contexts_from_case(self, occs, extraction_strategy: str) -> List[str]:
        contexts: List[str] = []
        if isinstance(occs, str):
            contexts.append(occs)
        elif isinstance(occs, dict):
            keys_to_check = ("context", "text", "snippet", "kwic", "content", "full_text", "body")
            if extraction_strategy == 'first':
                for k in keys_to_check:
                    if k in occs and isinstance(occs[k], str):
                        contexts.append(occs[k])
                        break
            else:  # 'all' or 'raw_json' (raw_json uses entire JSON elsewhere but for metrics we gather all)
                for k in keys_to_check:
                    v = occs.get(k)
                    if isinstance(v, str):
                        contexts.append(v)
        elif isinstance(occs, list):
            for o in occs:
                contexts.extend(self._extract_contexts_from_case(o, extraction_strategy))
        return contexts

    def _compute_stats(self, results_dict: Dict[str, Any], keyword: str, extraction_strategy: str) -> Dict[str, Any]:
        volumes = sorted(results_dict.keys(), key=lambda x: int(x) if str(x).isdigit() else str(x))
        case_counts = {}
        total_contexts = 0
        occurrences_per_case = []
        for vol, cases in results_dict.items():
            if not isinstance(cases, dict):
                continue
            case_counts[vol] = len(cases)
            for case_id, occs in cases.items():
                contexts = self._extract_contexts_from_case(occs, extraction_strategy)
                occ_count = len(contexts)
                total_contexts += occ_count
                occurrences_per_case.append({"volume": vol, "case_id": case_id, "occurrences": occ_count})
        return {
            "volumes": volumes,
            "case_counts": case_counts,
            "total_cases": sum(case_counts.values()),
            "total_contexts": total_contexts,
            "occurrences_per_case": occurrences_per_case,
            "keyword": keyword,
        }

    def _sample_contexts(self, results_dict: Dict[str, Any], max_contexts: Optional[int], extraction_strategy: str) -> List[str]:
        # If max_contexts is None/0/negative: include ALL contexts (no truncation)
        limit = max_contexts if isinstance(max_contexts, int) and max_contexts > 0 else None
        samples: List[str] = []
        if extraction_strategy == 'raw_json':
            return samples  # handled separately (we embed full JSON)
        for vol in sorted(results_dict.keys(), key=lambda x: int(x) if str(x).isdigit() else str(x)):
            cases = results_dict[vol]
            if not isinstance(cases, dict):
                continue
            for case_id, occs in cases.items():
                contexts = self._extract_contexts_from_case(occs, extraction_strategy)
                for ctx in contexts:
                    cleaned = ' '.join(ctx.split())  # no clipping
                    samples.append(f'[{vol}:{case_id}] {cleaned}')
                    if limit and len(samples) >= limit:
                        return samples
        return samples

    def _build_prompt(
        self,
        keyword: str,
        results_dict: Dict[str, Any],
        stats: Dict[str, Any],
        analysis_focus: str,
        max_contexts: Optional[int],
        return_json: bool,
        extraction_strategy: str,
        debug: bool,
        metrics: Dict[str, Any],
    ) -> str:
        if extraction_strategy == 'raw_json':
            raw_json_block = json.dumps(results_dict, indent=2)
            # Provide a short note, then raw JSON. Model must rely ONLY on raw JSON.
            contexts_section = (
                f"RAW_JSON_MODE: Entire filtered JSON provided below. Size chars={metrics['raw_chars']} approx_tokens={metrics['approx_tokens_raw']:.0f}.\n"
                "Do NOT hallucinate beyond this data.\n---\n" + raw_json_block + "\n---\n"
            )
            sample_contexts = []
        else:
            sample_contexts = self._sample_contexts(results_dict, max_contexts, extraction_strategy)
            if not sample_contexts:
                sample_contexts = ["(No context strings extracted — verify input JSON structure)"]
            contexts_section = (
                f"Sample Contexts ({len(sample_contexts)}) strategy={extraction_strategy} (max_contexts={max_contexts}):\n---\n"
                + "\n".join(sample_contexts) + "\n---\n"
            )

        focus_instructions = {
            "general": "Provide an overview of usage patterns, semantic ranges, and any interpretive variability.",
            "evolution": "Describe shifts across volumes (treat volume ordering as temporal proxy if applicable).",
            "judicial_philosophy": "Identify internal patterns that might hint at differing interpretive strategies (ONLY within provided data).",
            "custom": "Provide a comprehensive structured analysis (frequency, contextual clusters, potential senses).",
        }
        occ_lines = sorted(
            [f"{o['volume']}:{o['case_id']}={o['occurrences']}" for o in stats["occurrences_per_case"]],
            key=lambda x: x
        )[:80]

        debug_block = ""
        if debug:
            debug_block = (
                "DEBUG METRICS (for transparency, do NOT just repeat):\n"
                f"raw_chars={metrics['raw_chars']} extracted_chars={metrics['extracted_chars']} reduction_ratio={metrics['reduction_ratio']:.3f}\n"
                f"approx_tokens_raw={metrics['approx_tokens_raw']:.0f} approx_tokens_extracted={metrics['approx_tokens_extracted']:.0f} strategy={metrics['extraction_strategy']}\n"
            )

        base = f"""
            You are an AI analysis component of `getout_of_text_3`.
            STRICT RULE: Use ONLY the provided contexts / JSON. DO NOT introduce external cases, doctrines, or speculative references.
            Keyword: "{keyword}"
            Volumes: {', '.join(stats['volumes'])}
            Total Cases: {stats['total_cases']} | Total Context Snippets (computed with strategy='{extraction_strategy}'): {stats['total_contexts']}
            Occurrences Per Case (sample): {'; '.join(occ_lines)}
            Analysis Focus: {analysis_focus} → {focus_instructions.get(analysis_focus, focus_instructions['general'])}
            {debug_block}
            {contexts_section}
        """
        if return_json:
            base += (
                "Return ONLY valid JSON with this exact top-level structure (no extra prose):\n"
                "{\n"
                "  \"keyword\": string,\n"
                "  \"total_contexts\": number,\n"
                "  \"occurrences_summary\": string,\n"
                "  \"reasoning_content\": [string, ...],\n"
                "  \"summary\": string,\n"
                "  \"limitations\": string\n"
                "}\n"
                "Populate reasoning_content with a short step-by-step (3-6 bullets).\n"
                "If only one occurrence, reasoning_content should note insufficient data for variation."
            )
        else:
            base += (
                "Required Output Sections:\n1. Usage Summary\n2. Contextual Patterns / Proto-senses\n3. Frequency & Distribution Observations\n4. Interpretability Notes (ordinary meaning indicators)\n5. Open Questions / Ambiguities\nGround all claims ONLY in the contexts above."
            )
        if stats['total_contexts'] == 1:
            base += "\nNOTE: Only one occurrence detected."
        return base.strip()

    def _postprocess_json(self, content: str, results_dict: Dict[str, Any], stats: Dict[str, Any]) -> Dict[str, Any]:
        parsed = None
        try:
            parsed = json.loads(content)
        except Exception:
            if isinstance(content, str):
                match = re.search(r'{[\s\S]*}', content)
                if match:
                    try:
                        parsed = json.loads(match.group(0))
                    except Exception:
                        parsed = None
        if not isinstance(parsed, dict):
            parsed = {
                "keyword": stats['keyword'],
                "total_contexts": stats['total_contexts'],
                "occurrences_summary": f"{stats['total_contexts']} context snippet(s) across {stats['total_cases']} case(s)",
                "reasoning_content": [
                    "Model did not return valid JSON; wrapped raw text.",
                    "Single occurrence limits distributional inference." if stats['total_contexts']==1 else "Multiple contexts allow limited comparative analysis."
                ],
                "summary": content[:4000] if isinstance(content, str) else str(content)[:4000],
                "limitations": "Auto-wrapped due to invalid JSON from model."
            }
        for k, default in [
            ("reasoning_content", []),
            ("summary", ""),
            ("occurrences_summary", f"{stats['total_contexts']} snippet(s) across {stats['total_cases']} case(s)"),
            ("limitations", "")
        ]:
            if k not in parsed:
                parsed[k] = default
        return parsed

# Helper: format result so reasoning_content appears LAST under a heading
def format_result_output(result):
    """Return a single string with main answer first and reasoning_content appended at end.
    Format:
    <answer text>

    ## reasoning content
    ```text
    <reasoning>
    ```
    Handles several possible shapes returned by Bedrock / LangChain tool binding.
    """
    try:
        if isinstance(result, str):
            return result
        if isinstance(result, dict):
            reasoning = []
            rc = result.get("reasoning_content")
            if isinstance(rc, list):
                reasoning.append("\n".join(str(x) for x in rc))
            elif isinstance(rc, str):
                reasoning.append(rc)
            elif isinstance(rc, dict) and "text" in rc:
                reasoning.append(rc["text"])
            main_parts = []
            for key in ("summary", "text", "content"):
                if key in result and isinstance(result[key], str):
                    main_parts.append(result[key])
            main_text = "\n\n".join(p for p in main_parts if p and p.strip())
            reasoning_text = "\n\n".join(r for r in reasoning if r and r.strip())
            if reasoning_text:
                return f"{main_text}\n\n## reasoning content\n```text\n{reasoning_text}\n```" if main_text else f"## reasoning content\n```text\n{reasoning_text}\n```"
            return main_text or str(result)
        if isinstance(result, list):
            reasoning_segments = []
            answer_segments = []
            for block in result:
                if not isinstance(block, dict):
                    continue
                rc = block.get("reasoning_content")
                if isinstance(rc, dict) and "text" in rc and rc["text"].strip():
                    reasoning_segments.append(rc["text"].strip())
                elif isinstance(rc, str) and rc.strip():
                    reasoning_segments.append(rc.strip())
                if "text" in block and isinstance(block["text"], str) and block["text"].strip():
                    answer_segments.append(block["text"].strip())
            main_answer = "\n\n".join(answer_segments) if answer_segments else ""
            reasoning_text = "\n\n".join(reasoning_segments)
            if reasoning_text:
                return f"{main_answer}\n\n## reasoning content\n```text\n{reasoning_text}\n```" if main_answer else f"## reasoning content\n```text\n{reasoning_text}\n```"
            return main_answer or str(result)
        return str(result)
    except Exception as e:
        return f"(Formatting error: {e})\n{result}"
    
def export_markdown(result_text, keyword):
    """
    Export unified analysis (answer + reasoning content at end) to ONE markdown file.
    """
    safe_keyword = "".join(c if c.isalnum() or c in ("-", "_") else "_" for c in keyword.strip()) or "analysis_output"
    formatted_output = format_result_output(result_text)
    outfile = f"{safe_keyword}.md"
    with open(outfile, "w", encoding="utf-8") as f:
        f.write(formatted_output)
    print(f"Wrote combined analysis + reasoning to {outfile} (length={len(formatted_output)} chars)")
    return outfile


## Create the agent by binging the tools and the model together

Updated: We now have two tools bound:
1. `scotus_analysis` performs an internal search.
2. `scotus_filtered_analysis` ONLY analyzes pre-filtered JSON you already produced (no new search).

In [154]:
# Instantiate tools
scotus_tool = ScotusAnalysisTool(model=model, db_dict_formatted=db_dict_formatted)
filtered_scotus_tool = ScotusFilteredAnalysisTool(model=model)

tools = [scotus_tool, filtered_scotus_tool]
model_with_tools = model.bind_tools(tools)

print("Defined: ScotusAnalysisTool (searching) & ScotusFilteredAnalysisTool (pre-filtered with flexible parsing + optional JSON mode, normalized content)")
print(f" ✅ Tools bound to model: {[tool.name for tool in tools]}")
print(f" 👨‍⚖️ SCOTUS database has {len(db_dict_formatted)} volumes")

Defined: ScotusAnalysisTool (searching) & ScotusFilteredAnalysisTool (pre-filtered with flexible parsing + optional JSON mode, normalized content)
 ✅ Tools bound to model: ['scotus_analysis', 'scotus_filtered_analysis']
 👨‍⚖️ SCOTUS database has 242 volumes


### Compare extraction strategies (`first`, `all`, `raw_json`) with debug metrics
The next cell will:
1. Run a search for a moderately frequent keyword.
2. Invoke `scotus_filtered_analysis` three times with different `extraction_strategy` values.
3. Show debug metrics (raw vs extracted char counts, token estimates, reduction ratio).
4. Demonstrate preflight rejection if the raw JSON would exceed token limit (you can artificially force this by choosing a very frequent term or increasing context window upstream).

NOTE: `raw_json` embeds the entire JSON (can be huge) — use sparingly.


In [ ]:
_demo_keyword = 'vehicle'  # adjust to test token scaling
print(f"[DEMO] Building pre-filtered results for '{_demo_keyword}' (context_words=30)")
_demo_results = got3.search_keyword_corpus(
    keyword=_demo_keyword,
    db_dict=db_dict_formatted,
    case_sensitive=False,
    show_context=True,
    context_words=30,
    output='json'
)
# prune empties & sort
_demo_results = {k: v for k, v in sorted(_demo_results.items(), key=lambda item: int(item[0])) if v}
_demo_json = json.dumps(_demo_results)
print(f"Volumes with hits: {list(_demo_results.keys())[:8]} ... total_vols={len(_demo_results)}")

for strategy in ['first','all','raw_json']:
    print("\n============================")
    print(f"[DEMO] Strategy='{strategy}' (max_contexts=None, debug=True)")
    try:
        out = filtered_scotus_tool._run(
            keyword=_demo_keyword,
            results_json=_demo_json,
            analysis_focus='general',
            max_contexts=None,  # unlimited
            return_json=False,
            extraction_strategy=strategy,
            debug=True
        )
        # Only print a small slice to avoid flooding
        print(out[:600] + ('...' if len(out)>600 else ''))
    except Exception as e:
        print(f"(Error strategy={strategy}: {e})")
print("\n[DEMO] Extraction strategy comparison complete.")

## 1st tool: AI will search the corpus for you and analyze results

- for terms in the corpora that are very frequent, you'll almost certainly hit the `max_token` restriction (it's also frustrating that AWS Bedrock has limitations on the actual max tokens you can invoke versus what they publish is the maximum). To solve this, some recommendations:
    - batch the requests into multiple parts
    - lower the KWIC context window so you are sending less tokens
    - randomly drop some values
    - filter out stop words to cut down on char count

In [127]:
#keyword="bovine"
#keyword="dictionary"
#keyword="etienne"
#keyword="ordinary meaning"
#keyword="bank"
keyword="vehicle"

In [128]:
from time import time

search_keyword = keyword  # reuse previous, or set explicitly like: search_keyword = "dictionary"
analysis_focus = "general"  # options: general | evolution | judicial_philosophy | custom

print(f"[DEMO] Running scotus_analysis (live search) for keyword='{search_keyword}' focus='{analysis_focus}'")
start_time = time()
result_text = scotus_tool._run(keyword=search_keyword, analysis_focus=analysis_focus)
elapsed = time() - start_time

print(f"\n[DEMO] Runtime: {elapsed:.2f}s | Raw result type: {type(result_text).__name__}")
export_markdown(result_text, f"scotus_analysis_{search_keyword}")
# ============================================================================
# Unified formatted output (reasoning last)
#formatted = format_result_output(result_text)
#print("\n=== FORMATTED OUTPUT (reasoning at end) ===\n")
#print(formatted[:120])  # safety slice to avoid flooding notebook; adjust if needed

[DEMO] Running scotus_analysis (live search) for keyword='vehicle' focus='general'
🔍 TOOL(search): Searching SCOTUS database for keyword: 'vehicle'
🪄 AUTO-SHRINK: prompt 246322 > max_tokens 128000. context_words 20 -> 9 (ratio 0.520). Retrying...
📊 TOOL(search): Found 903 cases across 231 volumes | context_words_used=9 | shrink_attempts=1
🤖 TOOL(search): Sending 125359 characters to AI model for analysis
✅ TOOL(search): Analysis complete, returning 2 characters

[DEMO] Runtime: 37.27s | Raw result type: list
Wrote combined analysis + reasoning to scotus_analysis_vehicle.md (length=13522 chars)


'scotus_analysis_vehicle.md'

### Preview Markdown Report of AI summary

saved as `{keyword}.md`

## 2nd tool: Passing pre-filtered JSON results to the filtered analysis tool

- for greater control on the keyword samples

#### New parameters for `scotus_filtered_analysis`
- `extraction_strategy`: `first` (default) chooses first matching field among context/text/snippet/kwic/content/full_text/body; `all` concatenates all; `raw_json` embeds entire JSON (largest).
- `debug=True` adds transparency metrics (raw/extracted char counts, token estimates, reduction ratio) to stdout and prompt.
- Early preflight rejection triggers if the prompt's char/4 heuristic exceeds `max_tokens`.


In [170]:
keyword='bank'
loc_results = got3.search_keyword_corpus(
    keyword=keyword,
    db_dict=db_dict_formatted,
    case_sensitive=False,
    show_context=True,
    context_words=12,
    output="json"
)
# Drop keys with empty dicts and sort by keys (as integers)
filtered_sorted_results = {k: v for k, v in sorted(loc_results.items(), key=lambda item: int(item[0])) if v}
filtered_json_str = json.dumps(filtered_sorted_results)
# Plain text (narrative) mode
filtered_result_text = filtered_scotus_tool._run(
    keyword=keyword,
    results_json=filtered_json_str,
    analysis_focus='general',
    #max_contexts=40,
    return_json=False
)

🤖 TOOL(filtered): Sending 303280 chars (≈75820 tok) to model (contexts: 1964) | strategy=first return_json=False


In [171]:
print(filtered_result_text)
export_markdown(filtered_result_text, f"scotus_filtered_analysis_{keyword}_text")


{'type': 'reasoning_content', 'reasoning_content': {'text': 'We need to produce analysis of "bank" usage across many contexts. Provide sections. Base on snippets. Identify usage: noun meaning financial institution; also "bank" as riverbank; also "bank" as pool bank (bank of the pool); also "bank" as verb bank (to rely). Also "bank" in phrase "bankrupt" with hyphen? but focus "bank". Also "bank" as shelf, "bank" of a river, "bank" as "bank (finance)" vs other senses. Provide summary, patterns, frequency distribution (counts approximate). Provide interpretability notes and open questions.\n\nWe must only use contexts provided; no external knowledge. So we can infer senses from examples.\n\nLet\'s craft answer.', 'signature': ''}}
**1. Usage Summary**  
The 1 966 citations that contain the token **bank** fall overwhelmingly into two broad semantic families:  

| Sense (proto‑type) | Typical lexical environment (from the sample excerpts) | Rough share of total citations* |
|---------------

'scotus_filtered_analysis_bank_text.md'

______________________

## Conclusion 🐄 🐍 🧑‍⚖️

### AWS Bedrock + LangChain + `got3` + SCOTUS Corpus Analysis

This demo illustrated how to build custom AI‑powered analysis tools over a specialized legal text corpus using LangChain and AWS Bedrock. By combining dynamic keyword search with focused AI synthesis, we can rapidly explore judicial reasoning patterns in Supreme Court opinions.

### AWS Cost Explorer -- Monitoring Bedrock Costs

Useful `jq` filters for your AWS Cost Explorer output:

> notably costs won't show up immediately, so you may need to wait a day or two after incurring costs to see them reflected in Cost Explorer.

```bash
# Filter for Bedrock services specifically from all services with costs
named_profile=atn-developer
region=us-east-1
start_date=$(date +%Y-%m-01)
end_date=$(date +%Y-%m-%d)

aws ce get-cost-and-usage --time-period Start=$start_date,End=$end_date --granularity DAILY --metrics "BlendedCost" --group-by Type=DIMENSION,Key=SERVICE --region $region --profile=$named_profile | jq '.ResultsByTime[].Groups[] | select(.Keys[0] | test("Bedrock"; "i")) | {service: .Keys[0], cost: .Metrics.BlendedCost.Amount}'
```

#### Cost of AWS Bedrock

- on 10/01/2025 this was: `"cost": "0.03760425"` -- check tomorrow to see the costs from my testing runs today.
- okay various runs cost just about $0.10 `  "cost": "0.10061025"`

In [174]:
# Filter for Bedrock services specifically from all services with costs
!aws ce get-cost-and-usage --time-period Start=2025-09-30,End=2025-10-05 --granularity DAILY --metrics "BlendedCost" --group-by Type=DIMENSION,Key=SERVICE --region us-east-1 --profile=atn-developer | jq '.ResultsByTime[].Groups[] | select(.Keys[0] | test("Bedrock"; "i")) | {service: .Keys[0], cost: .Metrics.BlendedCost.Amount}'

{
  "service": "Amazon Bedrock",
  "cost": "0.10061025"
}
{
  "service": "Amazon Bedrock",
  "cost": "0.04100085"
}
{
  "service": "Amazon Bedrock",
  "cost": "0.0412656"
}
{
  "service": "Amazon Bedrock",
  "cost": "0.21341396"
}


## SCOTUS Analysis Tools (LangChain + `getout_of_text_3`)

This section documents two complementary LangChain Tool implementations for exploratory forensic / statutory‐interpretation analysis over a locally prepared SCOTUS corpus.

### 1. Purpose
You can (a) execute an on‑the‑fly keyword search + AI analysis, or (b) feed in *already filtered* JSON results for reproducible, cost‑controlled, deterministic re‑analysis with AI tools. This is for exploratory and demonstrative purposes only; do **NOT** rely on these tools for authoritative legal research or advice.

### 2. Tool Inventory
| Tool Name | Class | Performs Corpus Search? | Input Content Source | Best For | Cost / Token Control | Notes |
|-----------|-------|-------------------------|----------------------|----------|----------------------|-------|
| `scotus_analysis` | `ScotusAnalysisTool` | YES (internal `got3.search_keyword_corpus`) | Raw SCOTUS corpus dict (`db_dict_formatted`) | Quick ad‑hoc exploration, first look | Lower control (dynamic result size) | Returns free‑form model text. Not suited for strict reproducibility. |
| `scotus_filtered_analysis` | `ScotusFilteredAnalysisTool` | NO (analysis only) | Pre‑filtered JSON (already produced elsewhere) | Stable reports, batching, auditing, caching | High control (you decide slice + cap contexts) | Optional structured JSON output (`return_json=True`). |

### 3. Input Schemas (Pydantic)
1. `ScotusAnalysisInput`:
   - `keyword: str` – term/phrase to search.
   - `analysis_focus: str = 'general'` – one of: `general`, `evolution`, `judicial_philosophy`, `custom`.
2. `ScotusFilteredAnalysisInput`:
   - `keyword: str` – label only (no searching performed).
   - `results_json: str | dict` – JSON/dict from a *previous* `got3.search_keyword_corpus` call (after any manual filtering).
   - `analysis_focus: str = 'general'` – same options as above.
   - `max_contexts: int | None = None` – OPTIONAL cap; if None, 0, or <1 then ALL contexts are included (assumes you pre-trimmed).
   - `return_json: bool = False` – if `True` the prompt instructs the model to emit **strict JSON** (post‑validated / auto‑repaired if malformed).

### 4. Typical Workflow Patterns
| Scenario | Recommended Flow |
|----------|------------------|
| Rapid hypothesis check | Use `scotus_analysis` with a single keyword, review summary. |
| Iterative refinement / human curation | Run raw search once, manually prune / cluster results, then pass curated JSON to `scotus_filtered_analysis`. |
| Batch reporting (multiple keywords) | Precompute & persist each keyword’s JSON → loop over `scotus_filtered_analysis` with `return_json=True`. |
| Cost‑sensitive environment | Always pre‑filter & throttle with `max_contexts` (e.g. 40–80). |
| Need structured downstream ingestion | Use `return_json=True` and parse validated keys (`reasoning_content`, `summary`, etc.). |

### 5. Prompt Design Overview
Both tools:
- Enforce an “ONLY use supplied contexts” constraint (mitigates hallucination beyond current slice).
- Provide distribution metadata (volumes, counts, occurrence sample) to steer higher‑level synthesis.
- Offer `analysis_focus` to narrow stylistic / topical emphasis.

`scotus_analysis` additionally:
- Embeds a truncated JSON of search hits (first N characters) directly after retrieval.

`scotus_filtered_analysis` adds:
- Multi‑shape normalization for result structures: strings, lists, objects with `context`/`text` fields.
- Optional context sampling cap (set `max_contexts` > 0). If no cap is provided ALL contexts (full length) are included — no 240‑char clipping.
- Optional strict JSON response contract.

### 6. Output Shapes
| Mode | Example (abridged) |
|------|--------------------|
| `scotus_analysis` (text) | "Usage Summary... Contextual Patterns..." |
| `scotus_filtered_analysis` (`return_json=False`) | Same narrative section headings as above. |
| `scotus_filtered_analysis` (`return_json=True`) | `{ "keyword": "ordinary meaning", "total_contexts": 217, "occurrences_summary": "217 snippet(s)...", "reasoning_content": ["..."], "summary": "...", "limitations": "..." }` |

### 7. Reliability & Error Handling
- Async safety: attempts event‑loop reuse; falls back to sync if needed.
- JSON robustness: if model returns malformed JSON, a salvage regex pass wraps content into a valid fallback structure.
- Single‑occurrence safeguard: explicitly flags low‑evidence situations (e.g., only 1 context) to prevent over‑extrapolation.

### 8. Performance & Cost Notes
| Driver | Effect | Mitigation |
|--------|--------|------------|
| Large keyword result sets | Longer prompt → higher tokens | Pre‑filter & lower `max_contexts` |
| Very common terms (e.g., "the", functional words) | Noise / inflated contexts | Encourage user to refine / phrase search |
| High `max_contexts` + JSON mode | Larger instructions + response | Tune to 30–80; rarely need >100 contexts |
| Duplicate or near‑duplicate contexts | Redundant token usage | Consider de‑dupe preprocessing before passing JSON |

### 9. When NOT to Use These Tools
- You need full‑text semantic retrieval (vector / embedding) across the corpus → integrate an embedding + retriever pipeline instead.
- You require authoritative legal interpretation beyond provided snippets → domain attorney review required.
- You want cross‑corpus comparative linguistics (e.g., COCA vs SCOTUS) in one call → design a composite prompt / multi‑tool pipeline.

### 10. Minimal Usage Examples
Live search (exploratory):
```python
result_text = scotus_tool._run(keyword="textualism", analysis_focus="judicial_philosophy")
print(result_text[:800])
```
Pre‑filtered (deterministic):
```python
raw_results = got3.search_keyword_corpus(
    keyword="ordinary meaning",
    db_dict=db_dict_formatted,
    case_sensitive=False,
    show_context=True,
    context_words=30,
    output="json"
)
filtered = {k: v for k, v in raw_results.items() if v}  # prune empties
json_str = json.dumps(filtered)
analysis = filtered_scotus_tool._run(
    keyword="ordinary meaning",
    results_json=json_str,
    analysis_focus="general",
    max_contexts=50,
    return_json=True
)
analysis["summary"][:500]
```

### 11. Interpreting Structured JSON (Key Semantics)
| Key | Meaning | Typical Consumer Action |
|-----|---------|-------------------------|
| `keyword` | Echo label for downstream grouping | Index in dataframe / vector store |
| `total_contexts` | Sampled context count (post‑cap) | Assess evidence density |
| `occurrences_summary` | Human‑readable distribution summary | Display directly in UI |
| `reasoning_content` | Stepwise internal analysis (chain‑of‑thought lite) | Optional trust / audit pane |
| `summary` | Main synthesized narrative | Persist / compare across keywords |
| `limitations` | Self‑reported caveats (and JSON salvage note if any) | Flag for review / quality scoring |

### 12. Extension Ideas (Future Work)
- Add semantic clustering (MiniLM embeddings) before sampling to maximize diversity.
- Integrate rate limiting & token accounting dashboards.
- Provide a deterministic hash of the input JSON slice for reproducibility tracking.
- Optional KWIC alignment / colorized keyword highlighting inside samples.

### 13. Quick Decision Guide
> If you can already see and trust the filtered JSON you want analyzed, **use `scotus_filtered_analysis`**. Otherwise, start with `scotus_analysis` to discover whether the keyword is even worth a curated run.

---
**Reminder:** Both tools operate strictly over *your supplied corpus slice*. They intentionally do **NOT** fetch external case law or enrich with outside doctrinal knowledge. This keeps analyses auditable and grounded.
